In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import sympy as sym

import models
import symbolics

In [2]:
%matplotlib inline

# Maximizing equilibrium total offspring

In [3]:
x1, x2, x3, x4 = sym.symbols('x1, x2, x3, x4', real=True, nonnegative=True)
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)
M, m = sym.symbols("M, m", real=True, nonnegative=True)
epsilon = sym.symbols("epsilon", real=True, nonnegative=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

In [4]:
x = np.array([[x1], [x2], [x3], [1 - x1 - x2 - x3]])
payoff_kernel = np.array([[R, S], [T, P]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M, m, epsilon)
N, = models.total_offspring(W, x)

In [5]:
sym.factor(sym.cancel(sym.together(sym.expand(N))), UGA(x1 + x3), UgA(x1 + x3), x1, x2)

4.0*(0.5*P - 0.5*m + x1*(-0.5*M + 0.5*m) + x1*(-1.0*P + 0.5*S + 0.5*T)*UGA(x1 + x3) + x1*(1.0*P - 0.5*S - 0.5*T)*UgA(x1 + x3) + x1*(-0.5*P - 0.5*R + 0.5*S + 0.5*T)*UgA(x1 + x3)**2 + x1*(0.5*P + 0.5*R - 0.5*S - 0.5*T)*UGA(x1 + x3)**2 + x2*(-0.5*M + 0.5*m) + x2*(-1.0*P + 0.5*S + 0.5*T)*UGA(x1 + x3) + x2*(1.0*P - 0.5*S - 0.5*T)*UgA(x1 + x3) + x2*(-0.5*P - 0.5*R + 0.5*S + 0.5*T)*UgA(x1 + x3)**2 + x2*(0.5*P + 0.5*R - 0.5*S - 0.5*T)*UGA(x1 + x3)**2 + (-1.0*P + 0.5*S + 0.5*T)*UgA(x1 + x3) + (0.5*P + 0.5*R - 0.5*S - 0.5*T)*UgA(x1 + x3)**2)

Total offspring in the two-locus model is a convex combination of total offspring in the one-locus model where all females carry G-allele of the $\gamma$ gene and the one-locus model where all females carry the g-allele of the $\gamma$ gene.

\begin{align}
N(x_G, x_A; U_G, G_g) =& x_G \Bigg(2\bigg(\big((R + P) - (T + S)\big)U_G(x_A)^2 + \big((T + S) - 2P\big)U_G(x_A) + (P - M)\bigg)\Bigg) + (1 - x_G)\Bigg(2\bigg(\big((R + P) - (T + S)\big)U_g(x_A)^2 + \big((T + S) - 2P\big)U_g(x_A) + (P - m)\bigg)\Bigg) \\
=& x_G N(x_A; U_G) + (1 - x_G)N(x_A, U_g)
\end{align}

Note that the function for total offspring is linear in $x_G$. This fact implies that equilibrium total offspring will be maximized at either $x_G*=0$ or $x_G^*=1$, depending on parameters.  Thus any stable, fully polymorphic equilibrium will *not* maximize total offspring in equilibrium.

## Substitute equilibrium values

In [6]:
UGA_star, UgA_star, xG_star = sym.symbols("UGA_star, UgA_star, xG_star")
equilibrium_total_offspring = N.subs({UGA(x1+x3): UGA_star, UgA(x1+x3): UgA_star, x1: xG_star - x2}).simplify()

In [7]:
sym.factor(sym.cancel(equilibrium_total_offspring.subs({xG_star: 0, m: 0})), UgA_star)

4.0*(0.5*P + UgA_star**2*(0.5*P + 0.5*R - 0.5*S - 0.5*T) + UgA_star*(-1.0*P + 0.5*S + 0.5*T))

$$ N\bigg(\frac{1}{2\epsilon + 1}, 1, Ug\bigg) = \frac{2(R - M) + 4\epsilon\bigg(Ug^2\big((R + P) - (T + S)\big) + Ug\big((T + S) - 2P\big) + P\bigg)}{2\epsilon + 1} $$

## Make an interactive plot

In [25]:
_equilibrium_total_offspring = sym.lambdify((xG_star, UGA_star, UgA_star, T, R, P, S, M, m),
                                            equilibrium_total_offspring,
                                            modules="numpy")

In [39]:
def plot_total_offspring(xG_star, T, R, P, S, M):

    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    
    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    UGAs = equilibrium_selection_probs.reshape(-1, 1)
    UgAs = equilibrium_selection_probs.reshape(1, -1)
    Z = _equilibrium_total_offspring(xG_star, UGAs, UgAs, T, R, P, S, M, 0)
    cax = ax.imshow(Z, origin="lower")

    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^*$", fontsize=20)
    ax.set_title(r"Equilibrium max total offspring for $x_G^*$={} is {}".format(xG_star, Z.max()), fontsize=25)
    ax.grid(False)

    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [40]:
xG_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description=r"$x_G^*$")

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$M$")

w = widgets.interactive(plot_total_offspring, xG_star=xG_slider, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

interactive(children=(FloatSlider(value=0.5, description='$x_G^*$', max=1.0, step=0.01), FloatSlider(value=10.…

## Find the optimal values of $x_G^*, U_{GA}^*, U_{gA}^*$.

The number of total offspring can be written as a function of the equilibrium selection probability.

$$ N\big(x_G^*, U_{GA}^*, U_{gA}^*\big) = 2\bigg(\big((R + P) - (T + S)\big)x_G^*U_{GA}^{*2} + \big((T + S) - 2P\big)x_G^*U_{GA}^* + P - \big((R + P) - (T + S)\big)x_G^*U_{gA}^{*2} - \big((T + S) - 2P\big)x_G^*U_{gA}^* + \big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^* \bigg)$$


To find the equilibrium selection probability that maximizes the number of total offspring we need to solve the following constrained optimization problem.

$$ \max_{x_G^*, U_{GA}^*, U_{gA}^*}\ N\big(x_G^*, U_{GA}^*, U_{gA}^*\big)  $$

subject to the following inequality constraints.

\begin{align}
    -x_G^* \le& 0 \\
    x_G^* - 1 \le& 0 \\
    -U_{GA}^* \le& 0 \\
    U_{GA}^* - 1 \le& 0 \\
    -U_{gA}^* \le& 0 \\
    U_{gA}^* - 1 \le& 0
\end{align}

First-order conditions are as follows.
\begin{align}
    2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P - \big((R + P) - (T + S)\big)U_{gA}^{*2} - \big((T + S) - 2P\big)U_{gA}^*\bigg) =& -\mu_{x_G^*, 0} + \mu_{x_G^*,1} \\
    2\bigg(2\big((R + P) - (T + S)\big)x_G^*U_{GA}^* + \big((T + S) - 2P\big)x_G^*\bigg) =& -\mu_{U_{GA}^*, 0} + \mu_{U_{GA}^*,1} \\
    2\bigg(-2\big((R + P) - (T + S)\big)x_G^*U_{gA}^* - \big((T + S) - 2P\big)x_G^* + 2\big((R + P) - (T + S)\big)U_{gA}^* + \big((T + S) - 2P\big) \bigg) =& -\mu_{U_{gA}^*, 0} + \mu_{U_{gA}^*,1}
\end{align}

Complementary slackness conditions are

\begin{align}
    -\mu_{x_G^*,0}x_G^* =& 0 \\
    \mu_{x_G^*,1}\big(x_G^* - 1\big) =& 0 \\
    -\mu_{U_{GA}^*,0}U_{GA}^* =& 0 \\
    \mu_{U_{GA}^*,1}\big(U_{GA}^* - 1\big) =& 0
    -\mu_{U_{gA}^*,0}U_{gA}^* =& 0 \\
    \mu_{U_{gA}^*,1}\big(U_{gA}^* - 1\big) =& 0
\end{align}

where $\mu_0, \mu_1$ are Lagrange multipliers.

### Case 1: interior equilibrium $(0 < x_G^* < 1, 0 < U_{GA}^* < 1, 0 < U_{gA}^* < 1)$

In an interior equilibrium, complementary slackness conditions imply that all Lagrange multipliers are zero (i.e., $\mu_{x_G^*, 0} =\mu_{x_G^*, 1} =\mu_{U_{GA}^*, 0} =\mu_{U_{GA}^*, 0} =\mu_{U_{gA}^*, 0} =\mu_{U_{gA}^*, 0}=0$).

Our first order conditions reduce to the following.

\begin{align}
    2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P - \big((R + P) - (T + S)\big)U_{gA}^{*2} - \big((T + S) - 2P\big)U_{gA}^*\bigg) =& 0 \\
    2\bigg(2\big((R + P) - (T + S)\big)x_G^*U_{GA}^* + \big((T + S) - 2P\big)x_G^*\bigg) =& 0 \\
    2\bigg(-2\big((R + P) - (T + S)\big)x_G^*U_{gA}^* - \big((T + S) - 2P\big)x_G^* + 2\big((R + P) - (T + S)\big)U_{gA}^* + \big((T + S) - 2P\big) \bigg) =& 0
\end{align}

Rearranging the second first-order condition yields an expression for the optimal value of $U_{GA}^*$.

$$ \bar{U}_{GA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Substituting this result into the first first-order condition and rearranging yields an identical exprssion for the optimal value of $U_{gA}^*$.

$$ \bar{U}_{gA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Substituting this result into the third first-order condition yields a result which implies that the optimal value for $x_G^*$ is indeterminate (i.e., the objective is flat when holding $U_{GA}^*$ and $U_{gA}^*$ fixed). 


In [10]:
first_order_conditions = sym.Matrix([equilibrium_total_offspring.diff(xG_star, 1),
                                     equilibrium_total_offspring.diff(UGA_star, 1),
                                     equilibrium_total_offspring.diff(UgA_star, 1)])

In [11]:
optimal_UGA_star, = sym.solve(first_order_conditions[1,0], UGA_star)

In [12]:
optimal_UgA_star, = sym.solve(first_order_conditions[0,0].subs({UGA_star: optimal_UGA_star}), UgA_star)

In [13]:
# optimal value for xG_star is indeterminate!
sym.simplify(first_order_conditions[2,0].subs({UGA_star: optimal_UGA_star, UgA_star: optimal_UgA_star}))

0

In [14]:
jacobian = first_order_conditions.jacobian([xG_star, UGA_star, UgA_star])
simplified_jacobian = sym.simplify(jacobian.subs({UGA_star: optimal_UGA_star, UgA_star: optimal_UgA_star}))
e1, e2, e3 = (simplified_jacobian.eigenvals()
                                 .keys())

In [15]:
e1

-4*P*xG_star + 4*P - 4*R*xG_star + 4*R + 4*S*xG_star - 4*S + 4*T*xG_star - 4*T

In [16]:
e2

4*P*xG_star + 4*R*xG_star - 4*S*xG_star - 4*T*xG_star

In [17]:
e3

0

Requirement for total offspring to optimal a local maximum at the above values derived above is for the Hessian to be negative semi-definite. This requirement will be satisfied if and only if 

$$ R + P < T + S. $$

### Case 2: equilibrium with $\bar{x}_G^*=1$, $(0 < U_{GA}^* < 1, 0 < U_{gA}^* < 1)$

In this equilibrium, complementary slackness conditions imply that all Lagrange multipliers are zero (i.e., $\mu_{x_G^*, 0} =\mu_{U_{GA}^*, 0} =\mu_{U_{GA}^*, 0} =\mu_{U_{gA}^*, 0} =\mu_{U_{gA}^*, 0}=0$) except $\mu_{x_G^*, 1}  > 0$.

Our first order conditions reduce to the following.

\begin{align}
    2\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P - \big((R + P) - (T + S)\big)U_{gA}^{*2} - \big((T + S) - 2P\big)U_{gA}^*\bigg) =& \mu_{x_G^*, 1} \\
    2\bigg(2\big((R + P) - (T + S)\big)U_{GA}^* + \big((T + S) - 2P\big)\bigg) =& 0 \\
    2\bigg(-2\big((R + P) - (T + S)\big)U_{gA}^* - \big((T + S) - 2P\big) + 2\big((R + P) - (T + S)\big)U_{gA}^* + \big((T + S) - 2P\big) \bigg) =& 0
\end{align}

Rearranging the second first-order condition yields an expression for the optimal value of $U_{GA}^*$.

$$ \bar{U}_{GA}^* = \frac{1}{2}\left(\frac{2P - (T + S)}{(R + P) - (T + S)}\right) $$

Substituting this optimal value of $U_{GA}^*$ into the first first-order condition and rearranging we find that the inequality will hold so long as 

$$ \big((R + P) - (T + S)\big)\big(U_{gA}^* - \bar{U}_{GA}^*\big)^2 + P > 0 $$

which requires $R + P > T + S$. Finally, rearranging the third first-order condition implies that the optimal value for $U_{gA}^*$ is indeterminate: so long as $R + P > T + S$, then $\bar{x}_G^*=1$ for any value of $U_{gA}^*$.

In [62]:
def interior_optimal_UGA(T, R, P, S):
    return 0.5 * ((2 * P - (T + S)) / ((R + P) - (T + S)))


def interior_optimal_UgA(T, R, P, S):
    return interior_optimal_UGA(T, R, P, S)


def _mu_xG_1(UGA, UgA, T, R, P, S):
    multiplier = 2 * (((R + P) - (T + S)) * UGA**2 + ((T + S) - 2 * P) * UGA + P - 
                      ((R + P) - (T + S)) * UgA**2 - ((T + S) - 2 * P) * UgA)
    return multiplier


def max_total_fitness(T, R, P, S):
    if _mu_xG_1(UGA, UgA, T, R, P, S) > 0:
        pass # max at xG=1
    elif _mu_xG_1(UGA, UgA, T, R, P, S) < 0:
        pass # max at xG=0
    else:
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-62-fadb5b1c3254>, line 12)

In [69]:
objective = lambda x: -_equilibrium_total_offspring(x[0], x[1], x[2], 25, 3, 2, 1)
x0 = 0.5 * np.ones(3)
res = optimize.minimize(objective, x0, bounds=[(0,1), (0,1), (0,1)])

In [72]:
-res.fun

15.523809523809522